<a href="https://colab.research.google.com/github/itayBactizan/NLP-FinalProject/blob/main/date_code_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Get Parts (POS)

I marked in the code where I change the function 

In [ ]:
def get_parts(text, disp=False, verbose=True):
  global DEBUG
  doc = nlp(text.lower().translate(remove_punct_dict))
  docWithPunc = nlp(text.lower()) # itay change get text with punct
  date,time, gpe, ents =[], [], {}, [] # itay change add time
  action = ''
  multiple_cities = False
  if disp: p()
  for token in doc:
    if disp:
      p(f'{token.text}, dep_={token.dep_}, ent_type={token.ent_type_}, head={token.head}, lemma_={token.lemma_}, pos_={token.pos_}, tag_={token.tag_}')
    if token.ent_type_ == 'GPE': ents.append((token.text.lower(), token))
  for entity2 in docWithPunc.ents: # itay change
    if disp:
      print('with punc')
      print(f"{entity2.text}, {entity2.label_}='{spacy.explain(entity2.label_)}'")
    if entity2.label_=="TIME": time.append(entity2.text.lower()) # itay change 
    if entity2.label_ == 'DATE': date.append(entity2.text.lower()) # itay change
  for entity in doc.ents:
      if disp:
        print(f"{entity.text}, {entity.label_}='{spacy.explain(entity.label_)}', is_city={entity._.is_city}, is_country={entity._.is_country}")
      ent_text = entity.text.lower().translate(remove_punct_dict)  
      if entity.label_ == 'GPE':
        token = None
        for e in ents:
          if ent_text == e[0]:
            token = e[1] 
            break
        # verify the GPE is a true GPE for cases that a city has a name that has another meaninig
        if token and token.text != token.lemma_: # a true GPE cannot be lemmatized
          continue
        # for example there is a city in Denemark that has a name 'rain'
        if token and len(ents) > 1 and (token.head == token or token.pos_ not in ['PROPN', 'PRON', 'NOUN']) and not entity._.is_country:
          valid_gpe = False
          for e in ents:
            if ent_text != e[0]:
              tk = e[1]
              if tk.head == token:  # the token is connected to another gpe, so it is a valid gpe (for example, (paris, canada) are connected
                valid_gpe = True
                break
          if not valid_gpe:
            if DEBUG: p('A:', ent_text)
            continue
        ent_type = 'state'
        if DEBUG: p('0:', ent_text, entity._.is_city, entity._.is_country)
        if entity._.is_city and not entity._.is_country: # 'canada' is a country and also a city in Portugal
          ent_type = 'city'
          if ent_text in df_CITIES_API.index.values: 
            code_country = df_CITIES_API.loc[ent_text].country
            if type(code_country) != str:
              multiple_cities = True
              ent_country = countries_df.loc[code_country[0]].Name
            else:
              ent_country = countries_df.loc[code_country].Name
          elif ent_text in CITIES_IL:
            ent_country = 'israel'
          else:
            ent_country = ''
        elif entity._.is_country or ent_text in CITIES_API_country_code:
          ent_type = 'country'

        if ent_type not in gpe:
          gpe[ent_type] = []
        if ent_type == 'city':
           gpe[ent_type].append((entity.text, ent_country))
        else:
          gpe[ent_type].append(entity.text)
  if len(date) == 0:
    date = ['current']

  if DEBUG: p('1:', multiple_cities, gpe)
  if multiple_cities or ('city' in gpe and len(gpe['city']) > 1):
    if 'country' in gpe:
      # verify country code of the city is correct, when there multiple cities
      for i, (city, city_country) in enumerate(gpe['city']):
        country = gpe['country'][0]
        if city_country != country:
          for code_country in df_CITIES_API.loc[city].country:
            if city_country == countries_df.loc[code_country].Name:
              # found a matching country
              gpe['city'][i] = (city, country)
              break
    else:
      check_capital = True
      if len(gpe['city']) == 2: # maybe enter a country which has city with that name (eg. Toronto, US = there is a city name 'us' in france and 'usa' in japan)
        # 'city': [('toronto', 'Canada'), ('us', 'france')]
        for i in range(1,-1,-1):
          city = gpe['city'][i][0].upper()
          if city in countries_df.index: #usually the 2nd "city" is the country
            city_country = countries_df.loc[city].Name
            j = 0 if i == 1 else 1
            gpe['city'] = (gpe['city'][j][0], city_country)
            gpe['country'] = city_country
            check_capital = False
            break
      if check_capital:
        # check if one of the cities is a capital or a large city, then use it
        for i, (city, city_country) in enumerate(gpe['city']):
          if city in capitals_df.index.values:
            # found a matching country
            city_country = capitals_df.loc[city].country
            gpe['city'][i] = (city, city_country)
            break
          if city in largest_df.index.values:
            # found a matching country
            city_country = largest_df.loc[city].country
            gpe['city'][i] = (city, city_country)
            break
  elif 'country' in gpe and 'city' in gpe:  # country with city, verify correct
      # verify country code of the city is correct, when user entered 
      city, city_country = gpe['city'][0]
      country = gpe['country'][0]
      if city_country != country:
        gpe['city'] = [(city, f'ERROR: city [{city}] not found in country [{country}] - try a different spelling of the city')]
  elif 'country' in gpe and 'city' not in gpe:  # country without city, get the capital
    country = gpe['country'][0]
    if country in CAPITALS:
      gpe['city'] = [(CAPITALS[country], country)]
  result = {}
  result['date'] = date

  result['time']= time
  result.update(gpe)
  if disp:
    p(result)
  elif verbose and 'city' not in result:
    p(f'FAILED: org [{text}], {result}')
  return result

DEBUG = False
if False and TEST:
  get_parts("Today weather in a Ra-anana")
  get_parts("Today weather in a Ra'anana")  
  get_parts("Today weather in a Raanana")
  get_parts("weather in toronto") # there is a city toronto in USA and toronto is not the capital of Canada so we take the first city in the db
  get_parts("weather in toronto, canada")
  get_parts("weather paris next 5 days")
  get_parts("tell me how is the weather in the ra- anana in the")
  get_parts("rain in toronto")
get_parts("weather in toronto, us")
get_parts("weather in toronto, canada")
get_parts("weather in jenin")
get_parts("weather in jenin, israel")


{'city': [('jenin',
   'ERROR: city [jenin] not found in country [israel] - try a different spelling of the city')],
 'country': ['israel'],
 'date': ['current'],
 'time': []}

In [ ]:
from datetime import datetime

In [ ]:
from spacy.matcher import PhraseMatcher

In [ ]:
def load_dates():
  fname = 'datesWords.csv'
  if GIT:
    path_ = git_path
    print(path_)
  df = pd.read_csv(path_ + fname)
  return df
datesDf= load_dates()
datesDf.head()



```
# dictionary convert text to numbers
```



delta days by day of week 

In [ ]:
dayInWeek={'sunday':1,'monday':2,'tuesday':3,'Wednesday':4,'thursday':5,'friday':6,'saturday':7}

In [ ]:
def get_delta_days_weekday(text,dayInWeek ):
    days=-999
    week=-1
    dayInText=999
    todayDate=datetime.today()
    todayDayWeek=(todayDate.weekday()+2-7) if (todayDate.weekday()+2>7) else  (todayDate.weekday()+2) # get day of week today
    if text in list(dayInWeek.keys()): # if there is only one word of day in week
      dayInText=dayInWeek[text]
      if todayDayWeek<=dayInText:
          days=dayInText-todayDayWeek
      else:
          days=dayInText-todayDayWeek+7
    else:# if there are additional word like 'next sunday'
      doc=nlp(text)
      for entity in doc:
        print('the entity is  '+entity.text)
        if entity.text in list(dayInWeek.keys()):
            dayInText=dayInWeek[entity.text.lower()]
            if todayDayWeek<=dayInText:
                days=dayInText-todayDayWeek
            else:
                days=dayInText-todayDayWeek+7
                week=0
        if ((entity.lemma_=='next') & (week==-1)):
          print('get next')
          week=1
    if week==-1:
      week=0

    return days,week ,todayDayWeek,dayInText         


In [ ]:
get_delta_days_weekday("friday",dayInWeek)

In [ ]:
textToNumbers={'one':1,'two':2,'three':3,'four':4,'five':5,'six':6,'seven':7}

In [ ]:
def get_number_period(doc):
  week=0
  days=-999
  for entity in doc:
    print(entity.pos_)
    if entity.pos_=='NUM':
      print("found number "+entity.text)
      if (entity.text.isdigit()):
          num=int(entity.text)
      else:
          num=textToNumbers.get(entity.text,-999)
      print ('number is '+str(num))
      
      if (nlp(str(entity.head))[0].lemma_=='week'): # if there is week add it to number to week and if not match it num of days
        if (num!=-999):
          week=num
      elif (nlp(str(entity.head))[0].lemma_=='day') :
        days=num
  if ((week>0) &(days==-999)):
      days=0
  return days, week        

In [ ]:
get_number_period(nlp('in 2 day'))

fromat date function

In [ ]:
from dateutil.parser import parse ## added ver 2
from datetime import date ## added ver 2
import string ## added ver 2
p=print ## added ver 2
space_punct_dict = dict((ord(punct), ' ') for punct in string.punctuation) ## added ver 2

In [ ]:
## new function added ver 2
def is_valid_date_p(dt,disp=False):
    if dt:
        try:
            dt = dt.translate(space_punct_dict)
            new_date = str(parse(dt, fuzzy=True))[:10]
            if disp==True:
              print(new_date)
            year = int(new_date[:4])
            month = int(new_date[5:7])
            day = int(new_date[8:])
            ndays = (date(year, month, day) - date.today()).days
            return True, year, month, day, ndays
        except:
            p('invalid:', dt)
            return False, None, None, None, None
    return False

In [ ]:
# tests added ver 2
year, month, day = 2021, 7, 1
  
p(is_valid_date_p(f'{month}/{day}/{year}'))
p(is_valid_date_p(f'{month}.{day}.{year}'))
p(is_valid_date_p(f'{month},{day},{year}'))
p(is_valid_date_p(f'{month}/{day}/{year}'))
p(is_valid_date_p(f'{month}-{day}-{year}'))
p(is_valid_date_p(f'{month} {day} {year}'))
p()
p(is_valid_date_p('12/1/20'))
p(is_valid_date_p('12/31/20'))
p()
p(is_valid_date_p('31/12/20'))
p(is_valid_date_p('30/6/2020'))
p(is_valid_date_p('2020/30/6'))
p()
p(is_valid_date_p('june,30 2020'))
p(is_valid_date_p('today'))

In [ ]:
is_valid_date_p('3 chickens') # added ver 2

function for speial words like today,tommorow

In [ ]:
dateToNum={"today":0,"tomorrow":1,'twoDays':2 }

In [ ]:
# new function added ver 2
def get_days_special_words(df,doc,week,days,disp=False):
  numFinal=-999 
  for column in df: # run on each column if there is match
    matcher = PhraseMatcher(nlp.vocab)
    listNlp = [nlp.make_doc(t) for t in df[column].tolist() if str(t) != 'nan']
    matcher.add(column,None, *listNlp)
    matches = matcher(doc)
    print(matches)
    if len(matches)>0: # if we find match in the list of column
        print(column)
        for match_id, start, end in matches:
            if disp:
                print("matchFound",doc[start:end].text)
        if (column=="nextWeek"): # check if there 
            print("is next Week")
            week=1
        if days==-999:
          days=0                                      
          numFinal=dateToNum.get(column,-999)
        if disp==True:
          print("numFinal is "+str(numFinal))
        if (numFinal!=-999) | (days==-999):
          return numFinal,week
  return days,week

main function

In [ ]:
def check_date(df,dateText ,dateToNum,disp=False):
    rangeDays=1 # added ver 2
    days=-999
    days2=-999 # added ver 2
    week2=-999 # added ver 2
    week=0
    todayDayWeek=0 # get day in week today
    dayInText=0 # get day in week by text
    for text in dateText: # check if we get several date variables 
        if disp==True: # added ver 2
          print("date text "+ text) # added ver 2
        doc=nlp(text.lower()) # added ver 2
        days,week=get_days_special_words(df,doc,week,days,disp) # check special words which exist in the DF dates # added ver 2
        if(disp==True):          # added ver 2     
          print("1.days is "+str(days)+" and week is "+str(week)) # added ver 2                
        if ((week==0) & (days==-999) ): ## didn't find word in the DF dates # chaneg order  ver 2
          days,week,todayDayWeek,dayInText=get_delta_days_weekday(text.lower(),dayInWeek ) # check day in week and return num of days and week
          if(disp==True):# added ver 2
            print("2.days is "+str(days)+" and week is "+str(week)+" and days2 is "+str(days2)) # added ver 2
        if days2==-999:  # added ver 2
          days2,week2=get_number_period(doc) # check number of days/weeks "3 days", can be for two options, num of dasy and range # change order ver 2 
          if(disp==True): # added ver 2
            print("3.days is "+str(days)+" and week is "+str(week)+" and days2 is "+str(days2))# added ver 2
          if ((week==0) & (days==-999) & (days2==-999)): # check if get format # added ver 2
              print("check date format'")# added ver 2
              is_date, year, month, day, ndays=is_valid_date_p(text)# added ver 2
              if (disp==True) & (is_date==True):# added ver 2
                print("4.get date by format "+f"{year}-{month}-{day}" + " num of days is "+str(ndays))# added ver 2
              if is_date==True:# added ver 2
                days=ndays# added ver 2
                week=0# added ver 2
    print("number of range is  "+str(days2)+" number of dates" +str(days))# added ver 2        
    if (days2!=-999) & (days==-999):# added ver 2
      if disp==True:# added ver 2
        print("dates with just number of days/week")# added ver 2
      days=days2# added ver 2
      week=week2# added ver 2
    if (days2!=-999) & (days!=-999):# added ver 2
      if disp==True:# added ver 2
        print("get range")# added ver 2
      rangeDays=days2+ week2*7# added ver 2
    if (len(dateText)==1):# added ver 2
      rangeDays=1    # added ver 2
    if(disp==True):               # added ver 2   
      print("5.days is "+str(days)+" and week is "+str(week))# added ver 2
    if days!=-999:# added ver 2
      if ((todayDayWeek>dayInText) & (week==1)):
        week=0
      ndays= week*7+days# added ver 2
    else: # added ver 2
      ndays=-999 # added ver 2
    return ndays, rangeDays   # added ver 2  

In [ ]:
get_parts("what is the weather in sunday  next week for 3 days in paris")

In [ ]:
check_date(datesDf,["friday","next week","3 days"],dateToNum )